In [19]:
import json
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow.keras.layers as tfl
from tensorflow.keras.losses import BinaryCrossentropy
import tensorflow.keras.metrics as metrics
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.optimizers.schedules import CosineDecay
import time
from utils import *

In [20]:
print(f"TensorFlow / Keras Version: {tf.__version__}")

TensorFlow / Keras Version: 2.20.0


In [21]:
SEED=28
seed_everthing()
gpu_growth()
strategy = get_strategy()

For reproducibility, everything seeded!
Enabled memory growth for 1 GPU(s)
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Using GPU strategy: MirroredStrategy
REPLICAS: 1


In [22]:
AUTO = tf.data.AUTOTUNE
DATA_DIR = '../../../data/'
MODELS_DIR = '../../../models/'
IMAGE_SIZE = (256, 256)
SHUFFLE_SIZE = 1024
MASK_SIZE = IMAGE_SIZE
UNFREEZE_LAYER = 'conv5_block1_0_bn'
# === CELL 2: ALGORITHM CONFIGS ===
PENALIZED_F1_CONFIG = {'alpha_p': 0.5, 'stage': 3}
OPTIMIZATION_CONFIG = {'opt_trials': 20, 'opt_warmup_epoch': 3, 'opt_unfreeze_epoch': 5, 'unfreeze_layer': UNFREEZE_LAYER}
MODEL_CONFIG = {'img_size': IMAGE_SIZE, 'mask_size': MASK_SIZE}

BATCH_SIZE_PER_REPLICA = 8
GLOBAL_BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync
DATASET_CONFIG = {'shuffle': SHUFFLE_SIZE, 'batch_size': GLOBAL_BATCH_SIZE, 'auto': AUTO}
print(f'Global Batch size: {GLOBAL_BATCH_SIZE}')

Global Batch size: 8


In [23]:
class_mapping_dir = os.path.join(DATA_DIR, 'class_mapping.json')
with open(class_mapping_dir, 'r') as f:
    class_mapping = json.load(f)

print(class_mapping)

{'COVID': 0, 'Normal': 1, 'Viral Pneumonia': 2, 'Lung_Opacity': 3}


In [24]:
unhealthy_class_mapping_path = os.path.join(DATA_DIR, 'healthy_binary_mapping.json')
with open(unhealthy_class_mapping_path, 'r') as f:
    unhealthy_class_mapping = json.load(f)

print(unhealthy_class_mapping)

{'Healthy': 0, 'Unhealthy': 1}


In [25]:
all_files = sorted(tf.io.gfile.glob(os.path.join(DATA_DIR, 'tfrecords/*.tfrecord')))
sub_train_files = all_files[:3]
val_files = all_files[-1:]
print(
    f"Train files: {sub_train_files}\n"
    f"Val files: {val_files}"
)

Train files: ['../../../data/tfrecords/data_00.tfrecord', '../../../data/tfrecords/data_01.tfrecord', '../../../data/tfrecords/data_02.tfrecord']
Val files: ['../../../data/tfrecords/data_09.tfrecord']


In [26]:
parse_fn = make_parse_fn(image_size=IMAGE_SIZE, mask_size=MASK_SIZE)

In [27]:
light_augmentation = Sequential([
    tfl.Input(shape= IMAGE_SIZE + (3,)),
    tfl.RandomFlip('horizontal'),
    tfl.RandomRotation(0.05, interpolation='bilinear', fill_mode='nearest', seed=SEED),
    tfl.RandomZoom(0.05, interpolation='bilinear', fill_mode='nearest', seed=SEED),
    tfl.RandomBrightness(0.05, seed=SEED),
    tfl.RandomContrast(0.05, seed=SEED)
])

In [28]:
def dataset(tfrecords, config=DATASET_CONFIG, is_training= True, image_augmentation=None):
    shuffle_size = config["shuffle"]
    batch_size = config["batch_size"]
    AUTO = config["auto"]
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False
    dataset = tf.data.TFRecordDataset(tfrecords, num_parallel_reads= AUTO)
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(parse_fn, num_parallel_calls= AUTO)
    dataset = dataset.map(lung_roi_preprocess, num_parallel_calls=AUTO)
    dataset = dataset.map(remap_for_binary, num_parallel_calls=AUTO)
    
    if is_training:
        dataset = dataset.shuffle(shuffle_size)
        # 1. Batch the data FIRST
        dataset = dataset.batch(batch_size, drop_remainder= True)
        # 2. Apply augmentation to the entire batch SECOND
        if image_augmentation is not None:
            dataset = dataset.map(
                lambda x, y: (image_augmentation(x, training=True), y), 
                num_parallel_calls= AUTO
            )
    else:
        # For validation, just batch the data without augmenting
        dataset = dataset.batch(batch_size, drop_remainder= True)

    dataset = dataset.map(lambda x, y: (preprocess_input(x), y), num_parallel_calls= AUTO)
    # 3. Prefetch the augmented batches
    dataset = dataset.prefetch(AUTO)
    return dataset

In [29]:
sub_train_dataset = dataset(sub_train_files, is_training= True, image_augmentation=light_augmentation)
val_dataset = dataset(val_files, is_training= False, image_augmentation=light_augmentation)
print(f"Train and Validation datasets are created successfully !")

Train and Validation datasets are created successfully !


In [30]:
optuna_steps = count_steps(sub_train_dataset)
validation_steps = count_steps(val_dataset)
print(f"Optuna Training Steps: {optuna_steps}\nValidation Steps: {validation_steps}")

OPTIMIZATION_CONFIG['optuna_steps'] = int(optuna_steps)
OPTIMIZATION_CONFIG['validation_steps'] = int(validation_steps)

Optuna Training Steps: 793
Validation Steps: 264


In [31]:
best_arch_dir = './phase1_architecture/healthy_unhealthy-best_architecture.json'
with open(best_arch_dir, 'r') as f:
    best_arch = json.load(f)

print(f"Best Architecture metadata: {best_arch}")
print(
    f"Best Architecture Hyperparameters:\n"
    f"Number of Dense layers: {best_arch["best_hparams"]["num_layers"]}\n"
    f"Dense units in order: {best_arch["best_hparams"]["dense_units"]}"
)
OPTIMIZATION_CONFIG['phase1_hparams'] = best_arch['best_hparams']

Best Architecture metadata: {'best_trial_number': 12, 'best_value': 0.8881901409353111, 'best_hparams': {'num_layers': 2, 'dense_units': [1024, 512]}, 'phase1_settings': {'loss': 'BinaryCrossentropy', 'optimizer': 'Adam', 'lr': 0.0003, 'weight_decay': 0, 'dropout_rate': 0.1}, 'timestamp': 1768602873.5272806, 'seed': 28}
Best Architecture Hyperparameters:
Number of Dense layers: 2
Dense units in order: [1024, 512]


In [32]:
def optimization_hparams(trial, hparams=best_arch["best_hparams"]):
    num_layers = hparams["num_layers"]
    hparams["lr_stage1"] = trial.suggest_float("lr_stage1", 1e-4, 1e-3, log=True)
    hparams["lr_stage2"] = trial.suggest_float("lr_stage2", 5e-6, 5e-5, log=True)
    hparams["weight_decay"] = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
    hparams["label_smoothing"] = trial.suggest_categorical("label_smoothing", [0.0, 0.025, 0.05, 0.075, 0.1])
    hparams["dropout_rate"] = trial.suggest_float("dropout_rate", 0.1, 0.5, step=0.05)
    
    return hparams

In [33]:
def densenet_model(
    hparams, dropout_rate,
    config=MODEL_CONFIG
):
    
    img_size = config["img_size"]
    inputs = tfl.Input(shape= img_size + (3,))
    base_model = tf.keras.applications.DenseNet121(
        name= 'densenet',
        weights= 'imagenet',
        include_top= False
    )
    base_model.trainable = False
    for layer in base_model.layers:
        if isinstance(layer, tf.keras.layers.BatchNormalization):
            layer.trainable = False
    
    densenet = base_model(inputs, training= False)
    x = tfl.GlobalAveragePooling2D()(densenet)
    num_dense_layers = hparams["num_layers"]
    for i in range(num_dense_layers):
        units = hparams["dense_units"][i]
        x = tfl.Dense(units, activation= 'relu', name=f"head_dense_{i+1}")(x)
        dropout = tfl.Dropout(dropout_rate, name=f"dropout_{i+1}")
        x = dropout(x, training=True)

    outputs = tfl.Dense(1, activation= 'sigmoid')(x)
    model = tf.keras.Model(inputs, outputs)
    
    return model

In [34]:
def unfreeze_backbone(model, backbone_name= None, unfreeze_layer= None):
    base_model = model.get_layer(backbone_name)
    
    if unfreeze_layer is None:
        # Stage 1: Freeze everything
        base_model.trainable = False
        return model

    # Stage 2: Selective Unfreezing
    base_model.trainable = True
    unfreeze_flag = False
    
    for layer in base_model.layers:
        if layer.name == unfreeze_layer:
            unfreeze_flag = True
        
        if unfreeze_flag:
            # PROFESSIONAL RULE: Always keep BatchNormalization frozen during fine-tuning
            # to avoid destroying the moving mean/variance statistics.
            if isinstance(layer, tf.keras.layers.BatchNormalization):
                layer.trainable = False
            else:
                layer.trainable = True
        else:
            layer.trainable = False
            
    return model

In [35]:
def compile_model(model, loss, optimizer):
    model.compile(
        loss=loss,
        optimizer=optimizer,
        metrics=[
            metrics.BinaryAccuracy(name="accuracy"),
            metrics.Precision(name="precision"),
            metrics.Recall(name="recall"),
            metrics.AUC(name='AUC')
        ],
    )
    return model

In [36]:
def penalized_f1_score(history, config):

    
    """
    Your exact rolling window penalized F1 score function
    """
    alpha_p = config['alpha_p']
    stage_epochs = config['stage']

    val_prec = np.array(history.history["val_precision"])
    val_rec = np.array(history.history["val_recall"])
    val_f1 = np.array(2*(np.multiply(val_prec, val_rec)) / np.add(val_prec, val_rec))
    
    # Use last N epochs (adaptive for short architecture search)
    stage_epochs = min(stage_epochs, len(val_f1))
    f1 = np.mean(val_f1[-stage_epochs:])
    prec = np.mean(val_prec[-stage_epochs:])
    rec = np.mean(val_rec[-stage_epochs:])

    train_loss = history.history["loss"][-1]
    val_loss = history.history["val_loss"][-1]
    loss_penalty = alpha_p * (val_loss - train_loss)
        
    # Your gap penalty
    gap_penalty = alpha_p * abs(prec - rec)
    score = f1 - gap_penalty - loss_penalty
    
    return score, f1, prec, rec

In [37]:
def objective_optimization(trial, config=OPTIMIZATION_CONFIG):
    print('='*180)
    print(f"Trial {trial.number+1}/{config["opt_trials"]} started...")
    model = None
    history = None
    callbacks = None
    warmup_epoch = config['opt_warmup_epoch']
    unfreeze_epoch = config["opt_unfreeze_epoch"]
    total_epoch = warmup_epoch + unfreeze_epoch
    optuna_steps = config['optuna_steps']
    validation_steps = config['validation_steps']
    arch_hparams = config['phase1_hparams']
    unfreeze_layer = config["unfreeze_layer"]
    hparams = optimization_hparams(trial, hparams=arch_hparams)
    trial.set_user_attr("hparams", hparams)
    dropout_rate = hparams["dropout_rate"]
    loss = BinaryCrossentropy(
        label_smoothing=hparams['label_smoothing']
    )
    try:
        with strategy.scope():
                
            optimizer = AdamW(
                learning_rate=hparams["lr_stage1"],
                weight_decay=hparams["weight_decay"]
            )
            model = densenet_model(
                hparams, 
                dropout_rate=dropout_rate,
                config=MODEL_CONFIG
            )
            model = compile_model(model, loss=loss, optimizer=optimizer)
        history = model.fit(
                sub_train_dataset.repeat(),
                validation_data=val_dataset.repeat(),
                epochs=warmup_epoch,
                steps_per_epoch=optuna_steps,
                validation_steps=validation_steps,
            )
        with strategy.scope():
            model = unfreeze_backbone(
                model,
                backbone_name='densenet',
                unfreeze_layer=unfreeze_layer
            )
            decay_steps = total_epoch * optuna_steps
            lr = CosineDecay(
                decay_steps=decay_steps,
                initial_learning_rate=hparams["lr_stage2"],
                alpha=0.0
            )
            optimizer = AdamW(
                learning_rate=lr,
                weight_decay=hparams["weight_decay"]
            )
            model = compile_model(model, loss=loss, optimizer=optimizer)
            
        earlystop_cb = EarlyStopping(
            monitor='val_loss',
            patience=3,
        )
        callbacks = [earlystop_cb]
        history = model.fit(
            sub_train_dataset.repeat(),
            validation_data=val_dataset.repeat(),
            initial_epoch=warmup_epoch,
            epochs=total_epoch,
            steps_per_epoch=optuna_steps,
            validation_steps=validation_steps,
            callbacks=callbacks,
        )
        score, best_f1, best_prec, best_rec = penalized_f1_score(history, config=PENALIZED_F1_CONFIG)
        last_train_loss = history.history["loss"][-1]
        last_val_loss = history.history["val_loss"][-1]
        trial.set_user_attr("phase2_score", float(score))
        print(f"Penalized F1: {score:.4f}, Best F1: {best_f1:.4f}, P: {best_prec:.4f}, R: {best_rec:.4f}")
        print(f"Last Epoch --> Train Loss: {last_train_loss} | Val Loss: {last_val_loss}")
        return score
    finally:
        cleanup(model, history, callbacks)
        

In [38]:
#### Create an Optuna study object to find the best hyperparameter configuration
storage_dir = "sqlite:///phase2_optimization/healthy_unhealthy-phase2_optimization.db"
study = optuna.create_study(
    direction= 'maximize',
    storage=storage_dir,
    load_if_exists=True
)
# Start the optimization process over a fixed number of trials
study.optimize(lambda trial: objective_optimization(
    trial, config=OPTIMIZATION_CONFIG), 
    n_trials=OPTIMIZATION_CONFIG['opt_trials'], gc_after_trial= True)

best_trial = study.best_trial 
best_hparams = best_trial.user_attrs.get("hparams", None)

# collect metadata to save 
metadata = { 
    "best_trial_number": int(best_trial.number), 
    "best_value": float(best_trial.value), 
    "best_hparams": best_hparams, 
    "phase2_settings": { 
        "loss": "BinaryCrossentropy", 
        "optimizer": "AdamW", 
        "lr_schedules": "CosineDecay", 
        "warmup_epochs": OPTIMIZATION_CONFIG["opt_warmup_epoch"],
        "unfreeze_epochs": OPTIMIZATION_CONFIG["opt_unfreeze_epoch"],
        "unfreeze_layer": OPTIMIZATION_CONFIG["unfreeze_layer"],
        "trials": OPTIMIZATION_CONFIG["opt_trials"]
    }, 
    "timestamp": time.time(), 
    "seed": SEED
}

metadata_dir = "./phase2_optimization/healthy_unhealthy-best_hparams.json"
with open(metadata_dir, "w") as f:
    json.dump(metadata, f, indent=2)

[I 2026-01-21 14:30:02,705] A new study created in RDB with name: no-name-86f7d791-46ff-4a82-bec7-5f0dada29a1a


Trial 1/20 started...
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',)

2026-01-21 14:30:13.702402: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91300
2026-01-21 14:30:14.783474: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:382] Garbage collection: deallocate free memory regions (i.e., allocations) so that we can re-allocate a larger region to avoid OOM due to memory fragmentation. If you see this message frequently, you are running near the threshold of the available device memory and re-allocation may incur great performance overhead. You may try smaller batch sizes to observe the performance impact. Set TF_ENABLE_GPU_GARBAGE_COLLECTION=false if you'd like to disable this feature.


793/793 ━━━━━━━━━━━━━━━━━━━━ 116s 135ms/step - AUC: 0.8837 - accuracy: 0.8026 - loss: 0.4922 - precision: 0.8193 - recall: 0.7949 - val_AUC: 0.9292 - val_accuracy: 0.8333 - val_loss: 0.4608 - val_precision: 0.9464 - val_recall: 0.7050
Epoch 2/3
793/793 ━━━━━━━━━━━━━━━━━━━━ 105s 133ms/step - AUC: 0.9242 - accuracy: 0.8482 - loss: 0.4338 - precision: 0.8682 - recall: 0.8342 - val_AUC: 0.9365 - val_accuracy: 0.8542 - val_loss: 0.4301 - val_precision: 0.9396 - val_recall: 0.7555
Epoch 3/3
793/793 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - AUC: 0.9356 - accuracy: 0.8636 - loss: 0.4156 - precision: 0.8859 - recall: 0.8491

2026-01-21 14:35:10.780916: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: CANCELLED: Function was cancelled before it was started
	 [[{{node RemoteCall}}]]


...

793/793 ━━━━━━━━━━━━━━━━━━━━ 125s 158ms/step - AUC: 0.9667 - accuracy: 0.9105 - loss: 0.3257 - precision: 0.9236 - recall: 0.9022 - val_AUC: 0.9644 - val_accuracy: 0.9110 - val_loss: 0.3277 - val_precision: 0.9052 - val_recall: 0.9172
Epoch 8/8
793/793 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step - AUC: 0.9696 - accuracy: 0.9182 - loss: 0.3138 - precision: 0.9345 - recall: 0.9084

2026-01-21 19:49:52.610284: W tensorflow/core/kernels/data/prefetch_autotuner.cc:55] Prefetch autotuner tried to allocate 6291488 bytes after encountering the first element of size 6291488 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size
2026-01-21 19:49:53.268841: W tensorflow/core/kernels/data/prefetch_autotuner.cc:55] Prefetch autotuner tried to allocate 8388864 bytes after encountering the first element of size 8388864 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size
2026-01-21 19:49:53.268981: W tensorflow/core/kernels/data/prefetch_autotuner.cc:55] Prefetch autotuner tried to allocate 8388864 bytes after encountering the first element of size 8388864 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or 

793/793 ━━━━━━━━━━━━━━━━━━━━ 120s 152ms/step - AUC: 0.9708 - accuracy: 0.9171 - loss: 0.3136 - precision: 0.9306 - recall: 0.9080 - val_AUC: 0.9634 - val_accuracy: 0.9072 - val_loss: 0.3265 - val_precision: 0.9146 - val_recall: 0.8972
Penalized F1: 0.8831, Best F1: 0.9039, P: 0.9188, R: 0.8903
Last Epoch --> Train Loss: 0.3135804831981659 | Val Loss: 0.3265308439731598


[I 2026-01-21 19:50:50,768] Trial 19 finished with value: 0.8831489858358896 and parameters: {'lr_stage1': 0.0007220098212119819, 'lr_stage2': 2.6027633128633556e-05, 'weight_decay': 2.4975076576314156e-06, 'label_smoothing': 0.075, 'dropout_rate': 0.4}. Best is trial 10 with value: 0.8967769437770365.


🧹 RAM Cleaned. Current usage: 6032.79 MB
